In [1]:
import pandas as pd
from alive_progress import alive_bar
import xlwings as xw
import warnings

target_file = './结果表.xlsx'

df = pd.read_excel('./test.xlsx',sheet_name='Sheet1')
df

,地域,省份,产品年,数量,金额,市值
0,A,甲,2020,1,10,15
1,A,甲,2021,2,20,30
2,A,乙,2022,3,30,45
3,B,丙,2020,1,10,15
4,B,丁,2021,2,20,30
5,B,丁,2022,3,30,45
6,C,戊,2020,1,10,15
7,C,己,2021,2,20,30
8,C,己,2022,3,30,45


In [2]:
# 生成笛卡尔积

df_a = pd.DataFrame(df['地域'])
df_b = pd.DataFrame(df['省份'])
df_c = pd.DataFrame(df['产品年'])

df_a['临时列'] = 1
df_b['临时列'] = 1
df_c['临时列'] = 1
  
df_dikaer = pd.merge(df_a,df_b,on='临时列')
df_dikaer = pd.merge(df_dikaer,df_c,on='临时列')                    

df_dikaer = df_dikaer.drop('临时列',axis=1)
df_dikaer.drop_duplicates(inplace=True)
df_dikaer.head()

,地域,省份,产品年
0,A,甲,2020
1,A,甲,2021
2,A,甲,2022
18,A,乙,2020
19,A,乙,2021


In [3]:
# 关联（左连接）

df = pd.merge(df_dikaer,df,on=['地域','省份','产品年'],how='left').fillna(0)
df.head()

,地域,省份,产品年,数量,金额,市值
0,A,甲,2020,1.0,10.0,15.0
1,A,甲,2021,2.0,20.0,30.0
2,A,甲,2022,0.0,0.0,0.0
3,A,乙,2020,0.0,0.0,0.0
4,A,乙,2021,0.0,0.0,0.0


In [4]:
df_pivot = pd.pivot_table(df,index=['地域','产品年'],columns=['省份'],values=['数量','金额','市值'],aggfunc='sum', margins=True,fill_value=0)
df_pivot

市值                            数量        ...              金额          \
省份         丁   丙   乙   己   戊   甲    All  丁  丙  乙  ...  戊  甲   All   丁   丙   乙   
地域  产品年                                           ...                           
A   2020   0   0   0   0   0  15   15.0  0  0  0  ...  0  1   1.0   0   0   0   
    2021   0   0   0   0   0  30   30.0  0  0  0  ...  0  2   2.0   0   0   0   
    2022   0   0  45   0   0   0   45.0  0  0  3  ...  0  0   3.0   0   0  30   
B   2020   0  15   0   0   0   0   15.0  0  1  0  ...  0  0   1.0   0  10   0   
    2021  30   0   0   0   0   0   30.0  2  0  0  ...  0  0   2.0  20   0   0   
    2022  45   0   0   0   0   0   45.0  3  0  0  ...  0  0   3.0  30   0   0   
C   2020   0   0   0   0  15   0   15.0  0  0  0  ...  1  0   1.0   0   0   0   
    2021   0   0   0  30   0   0   30.0  0  0  0  ...  0  0   2.0   0   0   0   
    2022   0   0   0  45   0   0   45.0  0  0  0  ...  0  0   3.0   0   0   0   
All       75  15  45  75  15  45  270.0  5  1  3  ...  1  3  18.0  50  10  30   

                             
省份         己   戊   甲    All  
地域  产品年                      
A   2020   0   0  10   10.0  
    2021   0   0  20   20.0  
    2022   0   0   0   30.0  
B   2020   0   0   0   10.0  
    2021   0   0   0   20.0  
    2022   0   0   0   30.0  
C   2020   0  10   0   10.0  
    2021  20   0   0   20.0  
    2022  30   0   0   30.0  
All       50  10  30  180.0  

[10 rows x 21 columns]

In [5]:
df_subtotal = df_pivot.stack('省份').fillna(0).reset_index()
df_subtotal = df_subtotal[df_subtotal['地域']!='All']
df_subtotal['省份'] = df_subtotal['省份'].str.replace('All','小计')
df_subtotal

,地域,产品年,省份,市值,数量,金额
0,A,2020,小计,15.0,1.0,10.0
1,A,2020,丁,0.0,0.0,0.0
2,A,2020,丙,0.0,0.0,0.0
3,A,2020,乙,0.0,0.0,0.0
4,A,2020,己,0.0,0.0,0.0
...,...,...,...,...,...,...
58,C,2022,丙,0.0,0.0,0.0
59,C,2022,乙,0.0,0.0,0.0
60,C,2022,己,45.0,3.0,30.0
61,C,2022,戊,0.0,0.0,0.0


In [6]:
# 注意！此时“列”的纵向总计由于小计的存在，数值是实际值的 2 倍，需要后续处理（横向是无问题的）

df_pivot = pd.pivot_table(df_subtotal,index=['地域','省份'],columns=['产品年'],values=['数量','金额','市值'],aggfunc='sum',fill_value=0, margins=True,margins_name='总计')
df_pivot = df_pivot.drop(('总计','' ), axis=0)
df_pivot.loc[('总计','' ),:] = df_pivot.sum(axis=0) / 2
df_pivot

市值                       数量                    金额                   
产品年    2020  2021   2022     总计 2020 2021 2022    总计  2020  2021  2022     总计
地域 省份                                                                        
A  丁    0.0   0.0    0.0    0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0    0.0
   丙    0.0   0.0    0.0    0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0    0.0
   乙    0.0   0.0   45.0   45.0  0.0  0.0  3.0   3.0   0.0   0.0  30.0   30.0
   小计  15.0  30.0   45.0   90.0  1.0  2.0  3.0   6.0  10.0  20.0  30.0   60.0
   己    0.0   0.0    0.0    0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0    0.0
   戊    0.0   0.0    0.0    0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0    0.0
   甲   15.0  30.0    0.0   45.0  1.0  2.0  0.0   3.0  10.0  20.0   0.0   30.0
B  丁    0.0  30.0   45.0   75.0  0.0  2.0  3.0   5.0   0.0  20.0  30.0   50.0
   丙   15.0   0.0    0.0   15.0  1.0  0.0  0.0   1.0  10.0   0.0   0.0   10.0
   乙    0.0   0.0    0.0    0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0    0.0
   小计  15.0  30.0   45.0   90.0  1.0  2.0  3.0   6.0  10.0  20.0  30.0   60.0
   己    0.0   0.0    0.0    0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0    0.0
   戊    0.0   0.0    0.0    0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0    0.0
   甲    0.0   0.0    0.0    0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0    0.0
C  丁    0.0   0.0    0.0    0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0    0.0
   丙    0.0   0.0    0.0    0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0    0.0
   乙    0.0   0.0    0.0    0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0    0.0
   小计  15.0  30.0   45.0   90.0  1.0  2.0  3.0   6.0  10.0  20.0  30.0   60.0
   己    0.0  30.0   45.0   75.0  0.0  2.0  3.0   5.0   0.0  20.0  30.0   50.0
   戊   15.0   0.0    0.0   15.0  1.0  0.0  0.0   1.0  10.0   0.0   0.0   10.0
   甲    0.0   0.0    0.0    0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0    0.0
总计     45.0  90.0  135.0  270.0  3.0  6.0  9.0  18.0  30.0  60.0  90.0  180.0

In [7]:
# 自定义列字段的顺序（适用于多层索引的调换顺序、排序）

# 交换层级
df_pivot = df_pivot.reorder_levels([1,0],axis=1)

# # 自定义顺序
# 构建元组

cols_new = []

a = [2020,2021,2022,'总计']
b = ['数量','金额','市值']

for i in a:
    for m in b:
        list_temp = []
        list_temp.append(i)
        list_temp.append(m)
        cols_new.append(tuple(list_temp))
        
df_pivot = df_pivot[cols_new]

# df_pivot = df_pivot.sort_index(axis=1)
df_pivot = df_pivot.astype('float').reset_index()
df_pivot

产品年  地域  省份 2020             2021             2022                 总计         \
              数量    金额    市值   数量    金额    市值   数量    金额     市值    数量     金额   
0     A   丁  0.0   0.0   0.0  0.0   0.0   0.0  0.0   0.0    0.0   0.0    0.0   
1     A   丙  0.0   0.0   0.0  0.0   0.0   0.0  0.0   0.0    0.0   0.0    0.0   
2     A   乙  0.0   0.0   0.0  0.0   0.0   0.0  3.0  30.0   45.0   3.0   30.0   
3     A  小计  1.0  10.0  15.0  2.0  20.0  30.0  3.0  30.0   45.0   6.0   60.0   
4     A   己  0.0   0.0   0.0  0.0   0.0   0.0  0.0   0.0    0.0   0.0    0.0   
5     A   戊  0.0   0.0   0.0  0.0   0.0   0.0  0.0   0.0    0.0   0.0    0.0   
6     A   甲  1.0  10.0  15.0  2.0  20.0  30.0  0.0   0.0    0.0   3.0   30.0   
7     B   丁  0.0   0.0   0.0  2.0  20.0  30.0  3.0  30.0   45.0   5.0   50.0   
8     B   丙  1.0  10.0  15.0  0.0   0.0   0.0  0.0   0.0    0.0   1.0   10.0   
9     B   乙  0.0   0.0   0.0  0.0   0.0   0.0  0.0   0.0    0.0   0.0    0.0   
10    B  小计  1.0  10.0  15.0  2.0  20.0  30.0  3.0  30.0   45.0   6.0   60.0   
11    B   己  0.0   0.0   0.0  0.0   0.0   0.0  0.0   0.0    0.0   0.0    0.0   
12    B   戊  0.0   0.0   0.0  0.0   0.0   0.0  0.0   0.0    0.0   0.0    0.0   
13    B   甲  0.0   0.0   0.0  0.0   0.0   0.0  0.0   0.0    0.0   0.0    0.0   
14    C   丁  0.0   0.0   0.0  0.0   0.0   0.0  0.0   0.0    0.0   0.0    0.0   
15    C   丙  0.0   0.0   0.0  0.0   0.0   0.0  0.0   0.0    0.0   0.0    0.0   
16    C   乙  0.0   0.0   0.0  0.0   0.0   0.0  0.0   0.0    0.0   0.0    0.0   
17    C  小计  1.0  10.0  15.0  2.0  20.0  30.0  3.0  30.0   45.0   6.0   60.0   
18    C   己  0.0   0.0   0.0  2.0  20.0  30.0  3.0  30.0   45.0   5.0   50.0   
19    C   戊  1.0  10.0  15.0  0.0   0.0   0.0  0.0   0.0    0.0   1.0   10.0   
20    C   甲  0.0   0.0   0.0  0.0   0.0   0.0  0.0   0.0    0.0   0.0    0.0   
21   总计      3.0  30.0  45.0  6.0  60.0  90.0  9.0  90.0  135.0  18.0  180.0   

产品年         
        市值  
0      0.0  
1      0.0  
2     45.0  
3     90.0  
4      0.0  
5      0.0  
6     45.0  
7     75.0  
8     15.0  
9      0.0  
10    90.0  
11     0.0  
12     0.0  
13     0.0  
14     0.0  
15     0.0  
16     0.0  
17    90.0  
18    75.0  
19    15.0  
20     0.0  
21   270.0

In [8]:
# 自定义行字段的顺序（手动指定），自动指定用 sort_index()

type1 = pd.CategoricalDtype(categories=['A','B','C','总计'],ordered=True)
type2 = pd.CategoricalDtype(categories=['甲','乙','丙','丁','戊','己','小计',''],ordered=True)
df_pivot[('地域',   '')] = df_pivot[('地域',   '')].astype(type1)  #将指定列转成自定义的type
df_pivot[('省份',   '')] = df_pivot[('省份',   '')].astype(type2)
df_pivot.sort_values(by=[('地域',   ''),('省份',   '')],inplace=True)
df_pivot

产品年  地域  省份 2020             2021             2022                 总计         \
              数量    金额    市值   数量    金额    市值   数量    金额     市值    数量     金额   
6     A   甲  1.0  10.0  15.0  2.0  20.0  30.0  0.0   0.0    0.0   3.0   30.0   
2     A   乙  0.0   0.0   0.0  0.0   0.0   0.0  3.0  30.0   45.0   3.0   30.0   
1     A   丙  0.0   0.0   0.0  0.0   0.0   0.0  0.0   0.0    0.0   0.0    0.0   
0     A   丁  0.0   0.0   0.0  0.0   0.0   0.0  0.0   0.0    0.0   0.0    0.0   
5     A   戊  0.0   0.0   0.0  0.0   0.0   0.0  0.0   0.0    0.0   0.0    0.0   
4     A   己  0.0   0.0   0.0  0.0   0.0   0.0  0.0   0.0    0.0   0.0    0.0   
3     A  小计  1.0  10.0  15.0  2.0  20.0  30.0  3.0  30.0   45.0   6.0   60.0   
13    B   甲  0.0   0.0   0.0  0.0   0.0   0.0  0.0   0.0    0.0   0.0    0.0   
9     B   乙  0.0   0.0   0.0  0.0   0.0   0.0  0.0   0.0    0.0   0.0    0.0   
8     B   丙  1.0  10.0  15.0  0.0   0.0   0.0  0.0   0.0    0.0   1.0   10.0   
7     B   丁  0.0   0.0   0.0  2.0  20.0  30.0  3.0  30.0   45.0   5.0   50.0   
12    B   戊  0.0   0.0   0.0  0.0   0.0   0.0  0.0   0.0    0.0   0.0    0.0   
11    B   己  0.0   0.0   0.0  0.0   0.0   0.0  0.0   0.0    0.0   0.0    0.0   
10    B  小计  1.0  10.0  15.0  2.0  20.0  30.0  3.0  30.0   45.0   6.0   60.0   
20    C   甲  0.0   0.0   0.0  0.0   0.0   0.0  0.0   0.0    0.0   0.0    0.0   
16    C   乙  0.0   0.0   0.0  0.0   0.0   0.0  0.0   0.0    0.0   0.0    0.0   
15    C   丙  0.0   0.0   0.0  0.0   0.0   0.0  0.0   0.0    0.0   0.0    0.0   
14    C   丁  0.0   0.0   0.0  0.0   0.0   0.0  0.0   0.0    0.0   0.0    0.0   
19    C   戊  1.0  10.0  15.0  0.0   0.0   0.0  0.0   0.0    0.0   1.0   10.0   
18    C   己  0.0   0.0   0.0  2.0  20.0  30.0  3.0  30.0   45.0   5.0   50.0   
17    C  小计  1.0  10.0  15.0  2.0  20.0  30.0  3.0  30.0   45.0   6.0   60.0   
21   总计      3.0  30.0  45.0  6.0  60.0  90.0  9.0  90.0  135.0  18.0  180.0   

产品年         
        市值  
6     45.0  
2     45.0  
1      0.0  
0      0.0  
5      0.0  
4      0.0  
3     90.0  
13     0.0  
9      0.0  
8     15.0  
7     75.0  
12     0.0  
11     0.0  
10    90.0  
20     0.0  
16     0.0  
15     0.0  
14     0.0  
19    15.0  
18    75.0  
17    90.0  
21   270.0

In [9]:
# 姜两层列名恢复成单层

df_pivot.columns =[str(s1) +' '+ str(s2) for (s1,s2) in df_pivot.columns.tolist()]
df_pivot = df_pivot.reset_index(drop=True)
df_pivot

,地域,省份,2020 数量,2020 金额,2020 市值,2021 数量,2021 金额,2021 市值,2022 数量,2022 金额,2022 市值,总计 数量,总计 金额,总计 市值
0,A,甲,1.0,10.0,15.0,2.0,20.0,30.0,0.0,0.0,0.0,3.0,30.0,45.0
1,A,乙,0.0,0.0,0.0,0.0,0.0,0.0,3.0,30.0,45.0,3.0,30.0,45.0
2,A,丙,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,A,丁,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,A,戊,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,A,己,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,A,小计,1.0,10.0,15.0,2.0,20.0,30.0,3.0,30.0,45.0,6.0,60.0,90.0
7,B,甲,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,B,乙,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,B,丙,1.0,10.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,10.0,15.0


In [10]:
# 持久化输出

with pd.ExcelWriter(target_file) as writer:
    df_pivot.to_excel(writer,sheet_name='result', index=False)

app = xw.App(visible=False,add_book=False)
workbook = app.books.open('./结果表.xlsx')

with alive_bar(len(workbook.sheets)) as bar:    
    for i in workbook.sheets:
        bar()
        # pbar.set_description(f'Processing {i}')
    # 批量设置格式（行高、列宽、字体、大小、线框）
        value = i.range('A1').expand('table')# 选择要调整的区域
        value.rows.autofit() # 调整列宽字符宽度
        value.columns.autofit()  # 调整行高字符宽度
        value.api.Font.Name = '微软雅黑' # 设置字体
        value.api.Font.Size = 9 # 设置字号大小（磅数）
        value.api.VerticalAlignment = xw.constants.VAlign.xlVAlignCenter # 设置垂直居中
        value.api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter # 设置水平居中
        for cell in value:
            for b in range(7,12):
                cell.api.Borders(b).LineStyle = 1 # 设置单元格边框线型
                cell.api.Borders(b).Weight = 2 # 设置单元格边框粗细
        value = i.range('A1').expand('right')  # 选择要调整的区域
        value.api.Font.Size = 10
        value.api.Font.Bold = True  # 设置为粗体
        # print(f'《{i.name}》 页面处理完成……')
workbook.save()
workbook.close()
app.quit()

print(f'\n表格输出完成！！')

|████████████████████████████████████████| 1/1 [100%] in 9.4s (0.11/s) 

表格输出完成！！
